In [13]:
#!/usr/bin/python

# data processing packages
import numpy as np   
import pandas as pd 
import scipy as sp

import pylab

from pandas import *
from numpy import *
from scipy import *

import random
import sys

# machine leanring packages
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn import tree
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import precision_recall_fscore_support as score
from sklearn.model_selection import train_test_split
from sklearn import preprocessing

import xgboost as xgb

# own utilities
from utils_dataPrepro import *
# from ml_models import *
#from utils_keras import *

# statiscal models
import statsmodels as sm
from statsmodels.tsa.stattools import acf  
from statsmodels.tsa.stattools import pacf
from statsmodels.tsa.api import VAR, DynamicVAR

from statsmodels.stats import diagnostic


# visulization
%matplotlib inline    
import matplotlib as mplt
from matplotlib import cm
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import IPython
from IPython.display import display

In [ ]:
# GRANGER CAUSALITY

# http://davegiles.blogspot.ch/2011/04/testing-for-granger-causality.html
# https://menghublog.wordpress.com/2014/08/27/error-correction-model-in-time-series-regression/

# TO DO:
# consider the co-integration lag 


# Johansen's methodology (based on your VAR) for a reliable result
    
# In statistics, the Johansen test,[1] named after Søren Johansen, 
# is a procedure 
# for testing cointegration of several, say k, I(1) time series.

# Non-causality test 

# Seasonality test 

# Causality in seansonal data 

# http://davegiles.blogspot.ch/2015/03/granger-causality-seasonal-adjustment.html

# https://www.otexts.org/fpp/8/1

In [26]:
# dataset_str = str(sys.argv[1])
# print "Load dataset %s"%dataset_str

dataset_str = "pm25"

file_dic = { "stock":0,\
             "power":1,\
             "air":2,\
             "pm25":3
           }

files_list=[]
files_list.append( ["../../dataset/dataset_ts/stock_xtrain.dat", \
            "../../dataset/dataset_ts/stock_xtest.dat"] )
                    
files_list.append( ["../../dataset/dataset_ts/power_xtrain.dat", \
            "../../dataset/dataset_ts/power_xtest.dat"] )
                    
files_list.append( ["../../dataset/dataset_ts/air_xtrain.dat", \
            "../../dataset/dataset_ts/air_xtest.dat"] )

files_list.append( ["../../dataset/dataset_ts/pm25_xtrain.dat", \
            "../../dataset/dataset_ts/pm25_xtest.dat"] )


xtr = np.load(files_list[ file_dic[dataset_str] ][0])
xts = np.load(files_list[ file_dic[dataset_str] ][1])

# xtrain, xtest, _, _ = \
# prepare_train_test_data( False, files_list[ file_dic[dataset_str] ])

print np.shape(xtr), np.shape(xts)


(37800, 6) (5999, 6)


In [5]:
def ts_stationarize_diff( ts ):
    
    cur_ts = ts
    
    p_val = sm.tsa.stattools.adfuller(cur_ts,\
                                      regression='c', \
                                      maxlag=None, store=False)[1]
    diff_cnt = 0 
    while p_val > 0.01:
                
        pre_ts = cur_ts[1:]
        post_ts = cur_ts[:-1]
        
        diff_cnt += 1
        
        tmplen= len(pre_ts)
        
        cur_ts = [ pre_ts[i] - post_ts[i] for i in range(tmplen) ] 
        
        p_val = sm.tsa.stattools.adfuller(cur_ts,\
                                          regression='c', \
                                          maxlag=None, store=False)[1]
        
    return diff_cnt, p_val, cur_ts

def multi_ts_stationarize( dta ):
    
    post_len = []
    post_dta = []
    
    num_ts = len(dta[0])
    len_ts = len(dta)
     
    for i in range(num_ts):
        tmpts = [ dta[j][i] for j in range(len_ts) ]
        tmpts = ts_stationarize_diff( tmpts )
        
        print "stationary prepro: ", tmpts[0], tmpts[1], '\n'
        
        post_len.append( len(tmpts[2]) )
        post_dta.append( tmpts[2] )
        
    min_len = min(post_len)
        
    res_dta = []
    for i in range(num_ts):
        res_dta.append( post_dta[i][ post_len[i]-min_len : post_len[i] ] )
        
    return np.transpose(res_dta,[1, 0])

def causality_VAR(post_ts):
    
    model =  VAR(xtr)
    best_lag = model.select_order(60, verbose= False)
    
    print 'best lag: ', best_lag
    
    result = model.fit(best_lag['aic'])
    
    return result, best_lag

def causality_pairwise(VAR_res, post_ts):
    
    num_ts = len(post_ts[0])
    
    causing = []
    
    for i in range(num_ts):
        causing.append( [] )
        
        for j in range(num_ts):
            if j!=i:
                tmp_res = VAR_res.test_causality(i, [j], kind='wald', \
                                                verbose=False)
                
                if tmp_res['pvalue']< 0.01:
                    causing[-1].append(j)
    
    return causing

def causality_one(VAR_res, post_ts, ts_id):
    
    num_ts = len(post_ts[0])
    causing = []
        
    for j in range(num_ts):
        if j != ts_id:
            tmp_res = VAR_res.test_causality(ts_id, [j], kind='wald', \
                                                verbose=False)
                
            if tmp_res['pvalue']< 0.01:
                causing.append(j)
    
    return causing


In [28]:
post_ts = multi_ts_stationarize( xtr )
    
VAR_res,lag = causality_VAR(post_ts)
            
print causality_pairwise(VAR_res, post_ts)

stationary prepro:  0 1.52193882293e-07 

stationary prepro:  0 0.00342085835112 

stationary prepro:  0 7.8946082556e-11 

stationary prepro:  0 0.0 

stationary prepro:  0 0.0 

stationary prepro:  0 0.0 

best lag:  {'fpe': 54, 'hqic': 30, 'bic': 26, 'aic': 54}
[[1, 2, 5], [0, 2, 3, 5], [0, 1, 3, 5], [0, 1, 2], [5], [0, 1, 2]]


In [29]:
print causality_one(VAR_res, post_ts, 4)

[5]


In [20]:
pred_x = VAR_res.forecast(xtr[-lag['aic']:], 5)
print np.shape(pred_x)

(5, 8)


In [ ]:
# Causality test via VAR

def VAR_lag_selection( ts_df, max_lag):
    
    min_aic = np.inf
    min_lag = 0
    min_result = []
    
    for i in range(1,max_lag+1):
        model = VAR(ts_df)
        results = model.fit(i)
        
        aic = results.aic
        
        if aic<min_aic:
            min_lag = i
            min_aic = aic
            min_result = results
    
#     print min_result.summary()
    return min_lag, min_aic, min_result

# built-in lag order selection
model =  VAR(xtr)

best_lag = model.select_order(60, verbose= False)
print 'best lag: ', best_lag

result = model.fit(best_lag['aic'])


print result.test_causality(0, [1], kind='wald')

print '\n'

print result.test_causality(1, [0], kind='f')


In [ ]:
# Causality test via direct built-in routine  

# The Null hypothesis for grangercausalitytests is 
# that the time series in the second column, x2, 
# does NOT Granger cause the time series in the first column, x1. 

# The null hypothesis for all four test is that the 
# coefficients corresponding to past values of the second 
# time series are zero.

maxlag = 5
res = sm.tsa.stattools.grangercausalitytests(xtr, maxlag,\
                                addconst=True, verbose=False)

for i in range(1, maxlag+1):
    print res[i][0], '\n'